In [3]:
"""
- Your architecture should consist of a single hidden layer with up to k nodes.
- You can use any activation function (e.g., sigmoid, tanh, etc.) in the hidden nodes.
- Your model must use a bias term at the input and hidden layers. It can be a standalone term or
be incorporated in the weight matrices.
- You should use gradient descent to train your FFNN.
- You may find it helpful to use random number seeds for reproducibility when debugging.
- You do not need to use a GPU for this assignment, and your models should train in less than one
minute each.
- You are responsible for selecting hyperparameters (e.g., number of hidden nodes, learning rate,
training epochs, batch sizes, early stopping criteria, lambda, etc.). The goal is to get “good”
performance from your model, but an exhaustive hyper-parameter search is unnecessary.
- All code, exhibits and answers to free-response questions must be in a single Jupyter notebook.
- Your code should use parameters to control all functionality needed to complete specific tasks
(see below).
"""
import torch
import torch.nn as nn

class SingleLayerFFNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, activation_fn="sigmoid"):
        super(SingleLayerFFNN, self).__init__()

        self.input_layer = nn.Linear(input_size, hidden_size, bias=True)
        self.output_layer = nn.Linear(hidden_size, output_size, bias=True)

        if activation_fn == "sigmoid":
            self.activation = torch.sigmoid
        elif activation_fn == "tanh":
            self.activation = torch.tanh
        elif activation_fn == "relu":
            self.activation = torch.relu

    def forward(self, x):
        # One hidden layer
        hidden = self.activation(self.input_layer(x))
        output = self.output_layer(hidden)

        return output

In [ ]:
import torch.optim as optim

def train_model(model, train_loader, num_epochs, learning_rate):
    # nn.CrossEntropyLoss for multiclass cross entropy
    # nn.MSELoss for mean squared error
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        # loss for this epoch
        epoch_loss = 0
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader):.4f}")

In [ ]:
input_size = 2
# hidden_size from [2, 3, 5, 7, 9] 
hidden_size = 2
